<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
# normalize
# weights
# feed forward
# predict
# calc error
# back prop
# update weights based on gradient
# feed forward ...

In [20]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

class NeuralNet:
    def __init__(self, hidden_layers=1, epochs = 1000, verbose = 10, normalize = True):
        self.hidden_layers = hidden_layers
        self.input_scaler = MinMaxScaler()
        self.output_scaler = MinMaxScaler()
        self.epochs = epochs
        self.verbose = verbose
        self.normalize = normalize
        np.random.seed(420)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def normalize_data(self):
        self.norm_input = self.input_scaler.fit_transform(self.input)
        self.norm_output = self.output_scaler.fit_transform(self.output)
    
    def assign_output_nodes(self):
        self.output_nodes = self.output.shape[1]
        
    def assign_input_nodes(self):
        self.input_nodes = self.input.shape[1]
        
    def assign_hidden_nodes(self):
        self.hidden_nodes = self.input.shape[1] + 1
        
    def assign_weights(self):
        weights = []
        for ii in range(self.hidden_layers + 1):
            if ii == 0:
                # input to first hidden
                weights.append(np.random.rand(
                    self.input_nodes,
                    self.hidden_nodes
                ))
            elif ii == self.hidden_layers:
                # hidden to output layer
                weights.append(np.random.rand(
                    self.hidden_nodes,
                    self.output_nodes
                ))
            else:
                # hidden layers
                weights.append(np.random.rand(
                    self.hidden_nodes,
                    self.hidden_nodes
                ))
        self.weights = weights  
            
    def assign_all(self):
        self.assign_output_nodes()
        self.assign_input_nodes()
        self.assign_hidden_nodes()
        self.assign_weights()
        
    def feed_forward(self, inputs=None):
        if not inputs:
            inputs = self.input
        # TODO add hidden layers
        self.hidden_sum = np.dot(inputs, self.weights[0])
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        self.output_sum = np.dot(self.activated_hidden, self.weights[1])
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
    def backward(self, output):
        # TODO add hidden layers
        self.output_error = self.output - output
        self.output_delta = self.output_error * self.sigmoidPrime(output)
        
        self.z2_error = self.output_delta.dot(self.weights[1].T)
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        self.weights[0] += self.input.T.dot(self.z2_delta)
        self.weights[1] += self.activated_hidden.T.dot(self.output_delta)
                
    def fit(self, inputs, outputs):
        # TODO validation
        self.input = inputs
        self.output = outputs
        
        if self.normalize:
            self.normalize_data()
        
        self.assign_all()
        
        for ii in range(self.epochs):
            output = self.feed_forward()
            self.backward(output)
            if self.verbose and ((ii + 1) % self.verbose == 0 or ii == 0):
                print('+' + '---' * 3 + f'EPOCH {ii+1}' + '---'*3 + '+')
#                 print('Actual Output: \n', self.output)
#                 print('Predicted Output: \n', str(self.activated_output))
                print("Loss: \n", str(
                    np.mean(np.square(
                        self.output - self.activated_output
                    ))))
    
    def predict(self, pred_inputs):
        pred_outputs = self.feed_forward(pred_inputs)
        return (pred_outputs >= 0.5).astype(int)
        
        

In [21]:
nn = NeuralNet(epochs = 100000, verbose = 50000)

In [22]:
X = np.array([
    [0,0,1],
    [0,1,1],
    [1,0,1],
    [0,1,0],
    [1,0,0],
    [1,1,1],
    [0,0,0],
])
y = np.array([
    [0],
    [1],
    [1],
    [1],
    [1],
    [0],
    [0],
])

In [23]:
nn.fit(X, y)

+---------EPOCH 1---------+
Loss: 
 0.25510940810827515
+---------EPOCH 50000---------+
Loss: 
 4.19336086550674e-05
+---------EPOCH 100000---------+
Loss: 
 2.0919036186901414e-05


In [24]:
import pandas as pd

In [25]:
titanic = pd.read_csv('train.csv')

In [26]:
print(titanic.shape)
titanic.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
y = titanic['Survived'].values.reshape(-1, 1)

In [28]:
features = ['Pclass', 'SibSp', 'Parch', 'Fare']
X = titanic[features].values

In [29]:
nn = NeuralNet(epochs = 10000, verbose = 500)

In [30]:
nn.fit(X, y)

+---------EPOCH 1---------+
Loss: 
 0.508956905166524
+---------EPOCH 500---------+
Loss: 
 0.3838383838383838
+---------EPOCH 1000---------+
Loss: 
 0.3838383838383838


/Users/spentaur/.local/share/virtualenvs/lambda-e5gwxZ0x/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


+---------EPOCH 1500---------+
Loss: 
 0.3838383838383838
+---------EPOCH 2000---------+
Loss: 
 0.3838383838383838
+---------EPOCH 2500---------+
Loss: 
 0.3838383838383838
+---------EPOCH 3000---------+
Loss: 
 0.3838383838383838
+---------EPOCH 3500---------+
Loss: 
 0.3838383838383838
+---------EPOCH 4000---------+
Loss: 
 0.3838383838383838
+---------EPOCH 4500---------+
Loss: 
 0.3838383838383838
+---------EPOCH 5000---------+
Loss: 
 0.3838383838383838
+---------EPOCH 5500---------+
Loss: 
 0.3838383838383838
+---------EPOCH 6000---------+
Loss: 
 0.3838383838383838
+---------EPOCH 6500---------+
Loss: 
 0.3838383838383838
+---------EPOCH 7000---------+
Loss: 
 0.3838383838383838
+---------EPOCH 7500---------+
Loss: 
 0.3838383838383838
+---------EPOCH 8000---------+
Loss: 
 0.3838383838383838
+---------EPOCH 8500---------+
Loss: 
 0.3838383838383838
+---------EPOCH 9000---------+
Loss: 
 0.3838383838383838
+---------EPOCH 9500---------+
Loss: 
 0.3838383838383838
+---------EPOC

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [22]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [23]:
# input image dimensions
img_rows, img_cols = 28, 28

In [24]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [27]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [28]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [29]:
# Now the data should be in a format you're more familiar with
x_train.shape

(60000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [30]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp.reshape(-1, 1)

y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp.reshape(-1, 1)

In [31]:
# A Nice Binary target for ya to work with
y_train

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [32]:
x_train.shape

(60000, 784)

### Estimating Your `net

In [12]:
# https://towardsdatascience.com/building-an-artificial-neural-network-using-pure-numpy-3fe21acc5815

class RelU:
    def forward(self, inputs):
        # TODO think about other ways to do this
        return np.maximum(0, inputs)
    
    def backward(self, inputs, grad_input):
        
    
class Sigmoid:
    def forward(self, inputs):
        return 1/(1+np.exp(-inputs)) 
    

class Layer:
    def __init__(self, input_nodes, output_nodes, activation_function='relu'):
        self.input_nodes = input_nodes
        self.output_nodes = output_nodes
        self.weights = np.random.rand(
            self.input_nodes,
            self.output_nodes
        ) * 0.01
        self.bias = np.random.rand(output_nodes) * 0.01,
        # TODO shouldn't this be a property or something?
        self.activation_function = self.__set_activation(activation_function)
        
    def __set_activation(self, activation_function):
        functions = {
            'relu': RelU(),
            'sigmoid': Sigmoid()
        }
        return functions[activation_function]
        
    def forward(self, inputs):
        # TODO dot or matmul?
        summed_activation = np.dot(inputs, self.weights) + self.bias
        activation = self.activation_function.activate(summed_activation)
        self.activations = activations
        self.summed_activation = summed_activation
    
    def backward(self, inputs, grad_output):
        grad_input = np.dot(grad_output, self.weights.T)
        pass
    
    
class NeuralNetwork:
    def __init__(self, layers, learning_rate = 0.1):
        self.learning_rate = 0.1
        self.layers = []
        for layer in layers:
            self.layers.append(
                Layer(
                    layer['input_nodes'], 
                    layer['output_nodes'],
                    layer['activation_function']
                )
            )
    
    def __get_weights(self):
        weights = np.array([])
        for layer in self.layers:
            # TODO ravel vs flatten, speed/memory difference
            weights = np.append([weights, layer.weights.flatten()])
        return weights
    
    def __set_weights(self, weights):
        start = 0
        for idx, layer in enumerate(self.layers):
            stop = (layer.input_nodes * layer.output_nodes) + start
            layer_weights = np.reshape(weights[start:stop], 
                                       (layer.input_nodes, layer.output_nodes))
            start = stop
            self.layers[idx].weights = layer_weights
                
    
    def cost_function(self, predicted, reference):
        # https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795
        number_of_classes = predicted.shape[1]
        cost = (-1 / number_of_classes) * (
            np.dot(reference, np.log(predicted).T) + 
            np.dot(1 - reference, np.log(1 - predicted).T))
        return np.squeeze(cost)
    
    
    def cost_function_wrapper(self, weights, inputs, outputs):
        self.__set_weights(weights)
        predictions = self.predict(inputs)
        cost = self.cost_function(predictions, outputs)
    
    def optimize(self, inputs, outputs):
        weights = self.__get_weights()
        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(
            self.costFunctionWrapper,
            params0, 
            jac=True, 
            method='BFGS',
            args=(inputs, outputs), 
            options=options,
        )
    
    def fit(self, train_inputs, train_outputs):
        self.inputs = train_inputs
        self.outputs = train_outputs
        
    def forward(self, inputs):
        activations = inputs
        for layer in self.layers:
            activations = layer.forward(activations)
            
        return activations
    
    def backward(self):
        pass
    
    def train(self, train_inputs, train_outputs):
        # forward
        predictions = self.forward(train_inputs)
        cost = self.cost_function(predictions, train_outputs)
        cost_grad = self.cost_function_gradient(predictions, train_outputs)
        
        # backward
        for layer in reversed(self.layers[1:]):
            print(layer.input_nodes)
    
    def predict(self, inputs):
        return self.forward(inputs)
        

In [20]:
layers = [
    {
        'input_nodes': x_train.shape[1],
        'output_nodes': 200,
        'activation_function': 'relu'
    },
    {
        'input_nodes': 200,
        'output_nodes': 200,
        'activation_function': 'relu'
    },
    {
        'input_nodes': 200,
        'output_nodes': 2,
        'activation_function': 'sigmoid'
    }
]
nn = NeuralNetwork(layers)

In [21]:
nn.train(x_train, y_train)

(60000, 1) [[0.63776676]
 [0.65455256]
 [0.60024926]
 ...
 [0.61231473]
 [0.60379719]
 [0.60655136]]


IndexError: arrays used as indices must be of integer (or boolean) type

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?